In [1]:
import os
os.chdir("../../")

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import surgeon
import scanpy as sc

Using TensorFlow backend.


In [5]:
sc.settings.set_figure_params(dpi=200)

In [6]:
condition_key = "study"
cell_type_key = "cell_type"
target_conditions = ["Pancreas SS2", "Pancreas CelSeq2"]

In [7]:
adata = sc.read("./data/pancreas/pancreas_normalized.h5ad")
adata

AnnData object with n_obs × n_vars = 15681 × 1000 
    obs: 'batch', 'study', 'cell_type', 'size_factors'

In [9]:
source_adata = adata[~adata.obs[condition_key].isin(target_conditions)]
target_adata = adata[adata.obs[condition_key].isin(target_conditions)]

In [10]:
train_adata, valid_adata = surgeon.tl.train_test_split(source_adata, 0.80)
train_adata.shape, valid_adata.shape

((8235, 1000), (2059, 1000))

In [12]:
n_conditions = len(train_adata.obs[condition_key].unique().tolist())
n_conditions

3

In [13]:
network = surgeon.archs.CVAE(x_dimension=train_adata.shape[1], 
                             z_dimension=10,
                             architecture=[128, 20],
                             n_conditions=n_conditions,
                             lr=0.001,
                             alpha=0.0001,
                             beta=100,
                             eta=1.0,
                             clip_value=100,
                             use_batchnorm=False,
                             loss_fn='mse',
                             model_path="./models/CVAE/Kipoi/",
                             dropout_rate=0.05,
                             )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
conditions = adata.obs[condition_key].unique().tolist()
condition_encoder = surgeon.utils.create_dictionary(conditions, target_conditions)
condition_encoder

{'Pancreas inDrop': 0, 'Pancreas CelSeq': 1, 'Pancreas Fluidigm C1': 2}

In [17]:
network.train(train_adata,
              valid_adata, 
              condition_key=condition_key,
              le=condition_encoder, 
              n_epochs=1000,
              batch_size=512, 
              early_stop_limit=10,
              lr_reducer=8, 
              n_per_epoch=0,
              save=True, 
              retrain=True,
              verbose=5)

Instructions for updating:
Use tf.cast instead.
 |████----------------| 22.8%  - loss: 993.4934 - reconstruction_loss: 983.7499 - mmd_loss: 9.7435 - val_loss: 972.2976 - val_reconstruction_loss: 961.4778 - val_mmd_loss: 10.81991856

In [18]:
surgeon.kp.create_kipoi_model(model=network,
                              model_name="scNet-test",
                              upload=False,
                              data_url="https://zenodo.org/api/files/e05bed73-aa9b-4526-af26-e85d183d3f4d/pancreas_normalized.h5ad?versionId=126440ea-3ca1-4135-bea3-83f1694946ec",
                              data_md5="64fb9996af90b73592a5cf3690bac2ae2",
                              model_url="https://zenodo.org/api/files/e05bed73-aa9b-4526-af26-e85d183d3f4d/cvae.h5?versionId=6ef84515-a969-4f65-bdf9-1a863f321875",
                              model_md5="6ead4a0dde64d3678458400d1428966b"
                              )

Model YAML has been saved to /home/mohsen/projects/scnet/~/.kipoi/models/scNet-test
DataLoader YAML has been saved to /home/mohsen/projects/scnet/~/.kipoi/models/scNet-test
model is ready to submit!


In [ ]:
surgeon.kp.create_kipoi_model(model=network,
                              model_name="scNet",
                              upload=True,
                              data_path="https://zenodo.org/api/files/e05bed73-aa9b-4526-af26-e85d183d3f4d/pancreas_normalized.h5ad?versionId=126440ea-3ca1-4135-bea3-83f1694946ec",
                              access_token=""
                              )

## Congrats! Your model is ready to be added to Kipoi!
You have to fork this Github [__repository__](https://github.com/kipoi/models) before calling next function

```bash
git add MyModel/
git commit -m "Added <MyModel>"

git remote add fork https://github.com/<username>/models.git
git push fork master
```

## Finally!
Submit a pull-request on your forked repository